In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sqlite3
import string, re
import spacy

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import NMF
from sklearn.decomposition import LatentDirichletAllocation

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from collections import Counter

from corextopic import corextopic as ct
from corextopic import vis_topic as vt

import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/carolineshi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
allreviews_df = pd.read_pickle("fulldf.pkl")

In [3]:
allreviews_df['pos_reviews'] = allreviews_df.score.apply(lambda x: 1 if x >= 8.0 else 0)

In [4]:
allreviews_df['neg_reviews'] = allreviews_df.score.apply(lambda x: 1 if x <= 6.0 else 0)

In [5]:
positive_reviews_df = allreviews_df.loc[allreviews_df['pos_reviews'] == 1]

In [6]:
negative_reviews_df = allreviews_df.loc[allreviews_df['neg_reviews'] == 1]

In [7]:
positive_reviews_df

,id,review,processed,title,artist,url,score,best_new_music,author,author_type,pub_date,pub_weekday,pub_day,pub_month,pub_year,genre,label,year_released,pos_reviews,neg_reviews
0,22703,"“Trip-hop” eventually became a ’90s punchline,...",trip hop eventu becam punchlin press shorthand...,mezzanine,massive attack,http://pitchfork.com/reviews/albums/22703-mezz...,9.3,0,nate patrin,contributor,2017-01-08,6,8,1,2017,electronic,virgin,1998.0,1,0
3,22661,Kleenex began with a crash. It transpired one ...,kleenex began crash transpir one night long fo...,first songs,"kleenex, liliput",http://pitchfork.com/reviews/albums/22661-firs...,9.0,1,jenn pelly,associate reviews editor,2017-01-06,4,6,1,2017,rock,kill rock stars,2016.0,1,0
4,22725,It is impossible to consider a given release b...,imposs consid given releas footwork artist wit...,new start,taso,http://pitchfork.com/reviews/albums/22725-new-...,8.1,0,kevin lozano,tracks coordinator,2017-01-06,4,6,1,2017,electronic,teklife,2016.0,1,0
10,22715,Lots of drone musicians have been called sound...,lot drone musician call sound sculptor harri b...,clear sounds/perfetta,harry bertoia,http://pitchfork.com/reviews/albums/22715-clea...,8.0,0,marc masters,contributor,2017-01-04,2,4,1,2017,experimental,important,2016.0,1,0
11,22745,"On 2006’s “That’s Life,” Killer Mike boasted “...",life killer mike boast hard press find anoth r...,run the jewels 3,run the jewels,http://pitchfork.com/reviews/albums/22745-run-...,8.6,1,sheldon pearce,associate staff writer,2017-01-03,1,3,1,2017,rap,self-released,2016.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18389,1242,Remember back in elementary school when multim...,rememb back elementari school multimedia educ ...,califone,califone,http://pitchfork.com/reviews/albums/1242-calif...,9.2,0,mark richard-san,None,1999-02-18,3,18,2,1999,rock,flydaddy,1999.0,1,0
18390,1023,The ripples from Jawbox's dive into the underg...,rippl jawbox dive underground rock pond still ...,mission: control!,burning airlines,http://pitchfork.com/reviews/albums/1023-missi...,8.4,0,brent dicrescenzo,contributor,1999-02-16,1,16,2,1999,rock,desoto,NaN,1,0
18391,287,I got a heart so big it could crush this town....,got heart big could crush town realli swipe to...,all-in,arling and cameron,http://pitchfork.com/reviews/albums/287-all-in/,8.0,0,jason josephes,contributor,1999-02-11,3,11,2,1999,pop/r&b,emperor norton,NaN,1,0
18393,6455,"I'm psyched. Y'know why? Okay, okay, if you'...",psych know okay okay gonna drag guess tell mov...,sam prekop,sam prekop,http://pitchfork.com/reviews/albums/6455-sam-p...,8.9,0,ryan schreiber,editor-in-chief,1999-02-09,1,9,2,1999,electronic,thrill jockey,NaN,1,0


In [8]:
negative_reviews_df

,id,review,processed,title,artist,url,score,best_new_music,author,author_type,pub_date,pub_weekday,pub_day,pub_month,pub_year,genre,label,year_released,pos_reviews,neg_reviews
9,22724,There were innumerable cameos at the Bad Boy F...,innumer cameo bad boy famili reunion tour ofte...,filthy america its beautiful,the lox,http://pitchfork.com/reviews/albums/22724-filt...,5.3,0,ian cohen,contributor,2017-01-04,2,4,1,2017,rap,roc nation,2016.0,0,1
13,22720,"In January 2016, rapper/actor Yasiin Bey annou...",januari rapper actor yasiin bey announc depart...,december 99th,yasiin bey,http://pitchfork.com/reviews/albums/22720-dece...,3.5,0,marcus j. moore,contributor,2017-01-02,0,2,1,2017,rap,awge,2016.0,0,1
29,22708,Occultist and ceremonial magician Aleister Cro...,occultist ceremoni magician aleist crowley sai...,do what thou wilt.,ab-soul,http://pitchfork.com/reviews/albums/22708-do-w...,4.4,0,sheldon pearce,associate staff writer,2016-12-22,3,22,12,2016,rap,top dawg entertainment,2016.0,0,1
35,22678,"Technically, Fireplace: TheNotTheOtherSide is ...",technic fireplac thenottheothersid debut solo ...,fireplace: thenottheotherside,hodgy,http://pitchfork.com/reviews/albums/22678-fire...,5.0,0,matthew strauss,associate editor,2016-12-19,0,19,12,2016,rap,columbia,2016.0,0,1
41,22681,There aren’t many electronic dance musicians m...,mani electron danc musician cantanker beef leg...,w:/2016album/,deadmau5,http://pitchfork.com/reviews/albums/22681-w201...,5.9,0,philip sherburne,contributing editor,2016-12-16,4,16,12,2016,electronic,mau5trap,2016.0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18363,8732,"If this is what independent music has come to,...",independ come thing lot wors could ever imagin...,where's wolfie?,wolfie,http://pitchfork.com/reviews/albums/8732-where...,0.2,0,brent dicrescenzo,contributor,1999-03-23,1,23,3,1999,rock,parasol,NaN,0,1
18374,8598,Less really is more. Take the Wedding Present...,less realli take wed present exampl collect si...,singles 1989-1991,the wedding present,http://pitchfork.com/reviews/albums/8598-singl...,5.5,0,jason josephes,contributor,1999-03-09,1,9,3,1999,rock,manifesto,1999.0,0,1
18376,1610,"Really now, what the last remix album that was...",realli last remix worth anyon time bjork teleg...,cm: cornelius mix,cornelius,http://pitchfork.com/reviews/albums/1610-cm-co...,5.4,0,nick mirov,None,1999-03-09,1,9,3,1999,electronic,matador,1999.0,0,1
18385,2965,"Jason Falkner: Hello, my name is Jason, and I'...",jason falkner hello name jason former frontman...,can you still feel,jason falkner,http://pitchfork.com/reviews/albums/2965-can-y...,5.8,0,shan fowler,None,1999-02-23,1,23,2,1999,pop/r&b,elektra,NaN,0,1


In [93]:
negative_reviews_df.genre.value_counts()

rock            1986
electronic       330
rap              298
pop/r&b          228
folk/country      69
jazz              48
metal             45
experimental      43
global            21
Name: genre, dtype: int64

In [94]:
positive_reviews_df.genre.value_counts()

rock            1922
electronic       428
rap              275
pop/r&b          184
experimental     174
folk/country     131
jazz             105
metal            101
global            65
Name: genre, dtype: int64

## Positive Reviews -- Count Vectorizer

In [21]:
cv = CountVectorizer()

In [22]:
pos_words_cv = cv.fit_transform(positive_reviews_df['processed'])

In [23]:
pos_cv_df = pd.DataFrame(pos_words_cv.toarray(), columns = cv.get_feature_names())

In [12]:
pos_cv_df

,aa,aaa,aaaa,aaaaa,aaaaaah,aaah,aacm,aah,aaliyah,aam,...,zwan,zwei,zweit,zwickel,zydeco,zyg,zygot,zyx,zz,zzz
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3824,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3825,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3826,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3827,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Positive Reviews -- TF-IDF Vectorizer

In [99]:
tfidf_vectorizer = TfidfVectorizer()

In [100]:
pos_words_tfidf = tfidf_vectorizer.fit_transform(positive_reviews_df['processed'])

In [101]:
pos_tfidf_df = pd.DataFrame(pos_words_tfidf.toarray(), columns = tfidf_vectorizer.get_feature_names()).head(20)

## Positive Reviews -- TF-IDF Vectorizer x NMF Topic Modeling

In [102]:
nmf = NMF(n_components=10, random_state=42)

In [103]:
nmf_fit = nmf.fit(pos_words_tfidf)

/Applications/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:1076: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn("Maximum number of iterations %d reached. Increase it to"


In [50]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [105]:
display_topics(nmf, tfidf_vectorizer.get_feature_names(), 15)


Topic  0
love, song, sing, feel, voic, life, lyric, one, make, know, pop, way, record, get, girl

Topic  1
sound, record, guitar, electron, track, piec, drone, nois, instrument, work, loop, melodi, feel, minut, listen

Topic  2
rap, hop, hip, rapper, beat, sampl, mc, rhyme, jay, mixtap, product, kany, produc, vers, get

Topic  3
disc, version, dylan, record, set, song, releas, live, box, perform, includ, bowi, reissu, one, track

Topic  4
metal, black, doom, death, minut, riff, band, guitar, heavi, sunn, nois, fire, hardcor, agalloch, one

Topic  5
jazz, davi, mile, coltran, play, solo, parker, improvis, dejohnett, brew, set, corea, free, trumpet, quintet

Topic  6
danc, hous, disco, remix, track, techno, dj, mix, synth, pop, beat, dfa, sound, bass, dubstep

Topic  7
funk, motown, soul, label, gordi, brown, compil, hit, african, princ, africa, groov, horn, year, black

Topic  8
band, rock, punk, guitar, sound, pop, indi, record, one, get, song, even, make, post, great

Topic  9
beatl,

## Positive Reviews -- Corex Topic Modeling

In [24]:
words = list(np.asarray(cv.get_feature_names()))

In [25]:
topic_model = ct.Corex(n_hidden=10, words=words, seed=1)
topic_model.fit(pos_words_cv, words=words, docs=positive_reviews_df.processed)

In [27]:
topics = topic_model.get_topics()
for n,topic in enumerate(topics):
    topic_words,_,_ = zip(*topic)
    print('{}: '.format(n) + ','.join(topic_words))

0: reissu,version,would,peopl,life,earli,includ,career,later,decad
1: rap,fuck,rapper,hip,hop,shit,mc,rhyme,nigga,punk
2: electron,techno,song,ambient,synth,digit,sampl,loop,songwrit,lyric
3: drone,piec,compos,instrument,composit,minut,textur,feel,tone,begin
4: aor,mortal,languag,painstak,tea,scott,yard,crusad,pool,parlor
5: metal,death,black,howl,moment,dark,scream,blood,thrash,doom
6: funk,jazz,soul,african,brown,saxophonist,solo,west,africa,horn
7: domin,inadequ,keyboardist,horrifi,laetitia,sadier,commenc,joint,pretens,seasid
8: futur,king,entertain,check,stevi,articl,revolut,internet,friendli,consumpt
9: band,drummer,rock,guitarist,bassist,guitar,roll,youth,lineup,trio


## Negative Reviews -- Count Vectorizer

In [34]:
cv = CountVectorizer()

In [35]:
neg_words_cv = cv.fit_transform(negative_reviews_df['processed'])

In [36]:
neg_cv_df = pd.DataFrame(neg_words_cv.toarray(), columns = cv.get_feature_names())

In [32]:
neg_cv_df

,aa,aaa,aaaa,aaaaay,aaaahhhhhhkkkkghhh,aaaaiiighhhhyyyyeeee,aaah,aaahhhnnn,aaahk,aab,...,zydecali,zydeco,zygot,zygotik,zyklon,zyko,zynth,zz,zzzt,zzzz
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3446,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3447,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3448,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3449,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Negative Reviews -- TF-IDF Vectorizer

In [55]:
neg_tfidf_vectorizer = TfidfVectorizer()

In [56]:
neg_words_tfidf = neg_tfidf_vectorizer.fit_transform(negative_reviews_df['processed'])

In [57]:
neg_tfidf_df = pd.DataFrame(neg_words_tfidf.toarray(), columns = neg_tfidf_vectorizer.get_feature_names()).head(20)

In [58]:
neg_tfidf_df

,aa,aaa,aaaa,aaaaay,aaaahhhhhhkkkkghhh,aaaaiiighhhhyyyyeeee,aaah,aaahhhnnn,aaahk,aab,...,zydecali,zydeco,zygot,zygotik,zyklon,zyko,zynth,zz,zzzt,zzzz
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Negative Reviews -- TF-IDF Vectorizer x NMF Topic Modeling

In [59]:
neg_nmf = NMF(n_components=10, random_state=42)

In [60]:
neg_nmf_fit = neg_nmf.fit(neg_words_tfidf)

In [61]:
display_topics(neg_nmf, neg_tfidf_vectorizer.get_feature_names(), 15)


Topic  0
danc, pop, hous, disco, synth, punk, electro, singl, daft, funk, track, club, mobi, beat, french

Topic  1
rap, rapper, hop, hip, mixtap, beat, wayn, lil, vers, jay, produc, year, flow, rhyme, ross

Topic  2
sound, track, instrument, guitar, vocal, drum, record, minut, work, melodi, feel, seem, electron, bass, listen

Topic  3
remix, origin, ep, track, version, mix, one, dj, collect, materi, take, new, massiv, simon, hous

Topic  4
love, song, pop, songwrit, lyric, countri, voic, singer, folk, sing, sound, smith, adam, feel, piano

Topic  5
live, releas, disc, record, version, film, year, perform, two, compil, one, track, cover, materi, set

Topic  6
get, know, say, go, think, guy, thing, fuck, realli, good, one, peopl, make, want, girl

Topic  7
pollard, gbv, guid, sprout, bob, tobia, robert, record, voic, circu, captain, devil, superman, gentlemen, mouseman

Topic  8
band, rock, guitar, sound, group, indi, punk, song, riff, new, record, even, debut, post, choru

Topic  9
me

## Negative Reviews -- Corex Topic Modeling

In [37]:
words = list(np.asarray(cv.get_feature_names()))

In [ ]:
topic_model = ct.Corex(n_hidden=10, words=words, seed=1)
topic_model.fit(neg_words_cv, words=words, docs=negative_reviews_df.processed)

In [39]:
topics = topic_model.get_topics()
for n,topic in enumerate(topics):
    topic_words,_,_ = zip(*topic)
    print('{}: '.format(n) + ','.join(topic_words))

0: rap,rapper,hip,fuck,hop,shit,know,product,peopl,year
1: electron,textur,ambient,drum,percuss,drone,instrument,girl,synth,atmospher
2: lyric,songwrit,self,love,ballad,sing,pop,song,write,emot
3: smitten,siouxsi,zooey,crit,deschanel,smug,paparazzi,pere,yuppi,rhi
4: dave,cereal,bulletin,lo,vic,chesnutt,grohl,shadow,archivist,callback
5: banhart,squad,devendra,jam,omar,anthoni,atroci,wit,buffet,evil
6: back,album,play,black,wink,long,concern,aw,book,authent
7: sens,remain,emerg,arriv,toward,alway,given,record,follow,materi
8: trivial,neu,implod,cock,domest,infecti,stomp,portion,surg,kennedi
9: band,guitar,rock,guitarist,bassist,riff,metal,drummer,grung,frontman


## Comparing adjectives used in Positive vs Negative

In [42]:
pos_corpus = []
for text in positive_reviews_df['review']:
    pos_corpus.append(text)

In [44]:
tokenizer = RegexpTokenizer(r'(?u)\b[A-Za-z]+\b')

text = tokenizer.tokenize(str(pos_corpus))

In [45]:
pos_reviews_tagged = nltk.pos_tag(text)

In [72]:
adjs = [token[0] for token in pos_reviews_tagged if token[1] in ['JJ', 'JJR', 'JJS']]
pos_adj_frequency = nltk.FreqDist(adjs)

In [77]:
pos_adj_frequency.most_common(100)

[('n', 29105),
 ('s', 7457),
 ('first', 4089),
 ('more', 3970),
 ('new', 3889),
 ('other', 3235),
 ('t', 3185),
 ('own', 3146),
 ('most', 2631),
 ('few', 2441),
 ('best', 2425),
 ('much', 2416),
 ('same', 2344),
 ('many', 2340),
 ('last', 2238),
 ('good', 2155),
 ('full', 2005),
 ('early', 1902),
 ('little', 1778),
 ('old', 1697),
 ('great', 1671),
 ('single', 1645),
 ('such', 1571),
 ('second', 1556),
 ('vocal', 1379),
 ('musical', 1357),
 ('hard', 1330),
 ('live', 1274),
 ('different', 1253),
 ('long', 1237),
 ('high', 1174),
 ('r', 1159),
 ('big', 1097),
 ('acoustic', 1064),
 ('electronic', 1061),
 ('original', 1060),
 ('next', 1056),
 ('least', 1031),
 ('real', 1025),
 ('black', 1008),
 ('late', 959),
 ('easy', 941),
 ('heavy', 890),
 ('American', 880),
 ('whole', 878),
 ('classic', 873),
 ('better', 844),
 ('past', 836),
 ('simple', 835),
 ('recent', 794),
 ('clear', 786),
 ('less', 779),
 ('only', 756),
 ('short', 746),
 ('small', 737),
 ('final', 732),
 ('free', 729),
 ('perfect

In [63]:
neg_corpus = []
for text in negative_reviews_df['review']:
    neg_corpus.append(text)

In [64]:
tokenizer = RegexpTokenizer(r'(?u)\b[A-Za-z]+\b')

text = tokenizer.tokenize(str(neg_corpus))

In [65]:
neg_reviews_tagged = nltk.pos_tag(text)

In [66]:
adjs = [token[0] for token in neg_reviews_tagged if token[1] in ['JJ', 'JJR', 'JJS']]
neg_adj_frequency = nltk.FreqDist(adjs)

In [78]:
neg_adj_frequency.most_common(100)

[('n', 22488),
 ('s', 5206),
 ('more', 3007),
 ('t', 2890),
 ('new', 2520),
 ('first', 2342),
 ('other', 2213),
 ('much', 2185),
 ('same', 2093),
 ('good', 2093),
 ('best', 1968),
 ('few', 1965),
 ('little', 1888),
 ('own', 1876),
 ('most', 1739),
 ('least', 1603),
 ('last', 1538),
 ('many', 1474),
 ('old', 1342),
 ('such', 1221),
 ('full', 1177),
 ('hard', 1137),
 ('single', 1123),
 ('great', 1107),
 ('r', 1100),
 ('vocal', 1073),
 ('better', 999),
 ('second', 953),
 ('big', 895),
 ('musical', 880),
 ('high', 875),
 ('bad', 850),
 ('early', 849),
 ('real', 823),
 ('long', 801),
 ('acoustic', 776),
 ('original', 755),
 ('next', 691),
 ('past', 666),
 ('less', 659),
 ('live', 659),
 ('different', 654),
 ('only', 653),
 ('easy', 606),
 ('recent', 605),
 ('whole', 589),
 ('electronic', 579),
 ('interesting', 573),
 ('instrumental', 559),
 ('synth', 558),
 ('enough', 537),
 ('right', 537),
 ('low', 501),
 ('former', 488),
 ('late', 487),
 ('short', 482),
 ('true', 479),
 ('heavy', 477),
 (

## Comparing nouns used in Positve vs Negative

In [80]:
nouns = [token[0] for token in pos_reviews_tagged if token[1] in ['NN', 'NNS', 'NNP', 'NNPS']]
pos_noun_frequency = nltk.FreqDist(nouns)

In [82]:
pos_noun_frequency.most_common(50)

[('n', 20391),
 ('s', 18285),
 ('r', 13787),
 ('music', 10253),
 ('album', 9873),
 ('band', 7921),
 ('songs', 6249),
 ('song', 5575),
 ('time', 5228),
 ('record', 5154),
 ('t', 5012),
 ('rock', 4502),
 ('guitar', 4081),
 ('track', 4018),
 ('way', 3973),
 ('sound', 3782),
 ('tracks', 3589),
 ('pop', 3342),
 ('years', 3194),
 ('something', 2864),
 ('A', 2469),
 ('work', 2454),
 ('year', 2437),
 ('voice', 2227),
 ('world', 2145),
 ('sense', 1954),
 ('albums', 1927),
 ('vocals', 1926),
 ('people', 1892),
 ('bass', 1867),
 ('lyrics', 1755),
 ('kind', 1753),
 ('group', 1734),
 ('point', 1680),
 ('records', 1678),
 ('title', 1637),
 ('life', 1600),
 ('thing', 1599),
 ('New', 1598),
 ('end', 1584),
 ('part', 1583),
 ('minutes', 1561),
 ('debut', 1526),
 ('punk', 1524),
 ('release', 1454),
 ('things', 1427),
 ('label', 1413),
 ('sounds', 1410),
 ('My', 1402),
 ('line', 1394)]

In [83]:
neg_nouns = [token[0] for token in neg_reviews_tagged if token[1] in ['NN', 'NNS', 'NNP', 'NNPS']]
neg_noun_frequency = nltk.FreqDist(neg_nouns)

In [84]:
neg_noun_frequency.most_common(50)

[('n', 16111),
 ('s', 12483),
 ('r', 11998),
 ('album', 7312),
 ('band', 5742),
 ('music', 5020),
 ('songs', 4726),
 ('t', 4456),
 ('song', 4199),
 ('rock', 3762),
 ('record', 3475),
 ('time', 3212),
 ('track', 2909),
 ('pop', 2701),
 ('way', 2626),
 ('tracks', 2499),
 ('guitar', 2421),
 ('sound', 2249),
 ('something', 2100),
 ('years', 1975),
 ('lyrics', 1679),
 ('A', 1635),
 ('work', 1581),
 ('vocals', 1566),
 ('voice', 1447),
 ('kind', 1431),
 ('thing', 1420),
 ('year', 1401),
 ('debut', 1395),
 ('title', 1271),
 ('albums', 1212),
 ('point', 1211),
 ('sense', 1180),
 ('group', 1162),
 ('anything', 1145),
 ('minutes', 1125),
 ('people', 1104),
 ('sort', 1086),
 ('nothing', 1079),
 ('world', 1070),
 ('production', 1068),
 ('things', 1061),
 ('lot', 1011),
 ('moments', 987),
 ('line', 978),
 ('end', 978),
 ('part', 976),
 ('Love', 976),
 ('bit', 969),
 ('My', 965)]